# **Connect Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd '/path-your-dataset-folder/'

#**Install Module**

In [ ]:
!pip install dicttoxml 

#**COCO to Pascal VOC**

**Functions** 
* I used this code for doing segmentation my own Coco format dataset. It has one object every image. So if you want to use another dataset that has more than one object in one image, you have to add new object label in xml file for every object. Also data load part must change too. 
* If you don't want to do segmentation, you change "segmented" key from 1 to 0.

In [1]:
def create_pascal(folder, filename,h,w,cat, x_min,x_max, y_min, y_max):
  new_dict = {}
  source ={"database": "Unknown"}
  size = {"width": w,
          "height":h,
          "depth": 3}
  box = {"xmin": x_min,
         "ymin": y_min,
         "xmax": x_max,
         "ymax": y_max}
  obj = {"name": cat,
         "pose": "Unspecified",
         "truncated": 0,
         "difficult": 0,
         "bndbox": box}
  annotation = {"folder": folder,
                "file_name": filename,
                "path":folder+"/"+filename,
                "source":source,
                "size":size,
                "segmented":1,
                "object":obj}
  return annotation

In [ ]:
def get_category(ann, cat):
  id_ = ann["category_id"]
  for i in cat:
    if i["id"] ==id_:
      label = i["name"]
  return label

Load COCO Format Dataset

In [ ]:
import json 

path = "/your-json-file.json"
f = open(path)
data = json.load(f)

If your COCO format dataset has folder name in "*file_name*" key, you delete *folder* variable 

In [ ]:
from dicttoxml import dicttoxml
from xml.dom.minidom import parseString

for i, ann in enumerate(data["annotations"]):
  x1,y1,x_w,y_h = ann["bbox"]
  x2 = x1 + x_w
  y2 = y1 + y_h
  label = get_category(ann,data["categories"])
  h = data["images"][i]["height"]
  w = data["images"][i]["width"]
  filename = data["images"][i]["file_name"]
  folder = "valid"
  last_d = create_pascal(folder,filename,h,w,label,x1,x2,y1,y2)
  xml_ = dicttoxml(last_d, custom_root='annotation', attr_type=False)
  n,_ = filename.split(".")
  with open("/path-to-save/pascalvoc/valid/"+n+".xml", "w") as f:
    f.write(parseString(xml_).toprettyxml())
  f.close()